In [13]:
# Import necessary libraries
import smtplib
import ssl
from email.message import EmailMessage
import requests

# Define the get_direct_download_link function
def get_direct_download_link(url):
    # For Google Drive share links
    if 'drive.google.com' in url:
        if '/d/' in url:
            file_id = url.split('/d/')[1].split('/')[0]
        elif 'id=' in url:
            file_id = url.split('id=')[1]
        else:
            return url
        return f'https://drive.google.com/uc?export=download&id={file_id}'
    return url

# Define the necessary configuration variables
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 465
MY_EMAIL = 'nnilayy.work@gmail.com'        # Replace with your email address
MY_PASSWORD = 'pifu utzt tmvz zdyz'        # Replace with your email password or app password
VERCEL_APP_URL = 'https://your-vercel-app.vercel.app'  # Replace with your Vercel app URL

# Define the send_email_with_attachments function
def send_email_with_attachments(to_email, subject, body, resume_url=None, cover_letter_url=None):
    try:
        # Create the email message
        msg = EmailMessage()
        msg['Subject'] = subject
        msg['From'] = MY_EMAIL
        msg['To'] = to_email
        msg.set_content(body, subtype='html')

        # Embed tracking pixel
        tracking_pixel_url = f'{VERCEL_APP_URL}/api/track?email={to_email}'
        tracking_pixel = f'<img src="{tracking_pixel_url}" width="1" height="1" style="display:none;">'
        msg.add_alternative(f'{body}{tracking_pixel}', subtype='html')

        # Attach Resume
        if resume_url:
            direct_resume_url = get_direct_download_link(resume_url)
            response = requests.get(direct_resume_url)
            response.raise_for_status()
            resume_content = response.content
            msg.add_attachment(resume_content, maintype='application', subtype='pdf', filename='Resume.pdf')

        # Attach Cover Letter
        if cover_letter_url:
            direct_cover_letter_url = get_direct_download_link(cover_letter_url)
            response = requests.get(direct_cover_letter_url)
            response.raise_for_status()
            cover_letter_content = response.content
            msg.add_attachment(cover_letter_content, maintype='application', subtype='pdf', filename='CoverLetter.pdf')

        # Send the email
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT, context=context) as server:
            server.login(MY_EMAIL, MY_PASSWORD)
            server.send_message(msg)
        print(f"Email sent to {to_email}")

    except Exception as e:
        print(f"Failed to send email to {to_email}. Error: {e}")

# Now, call the function with hardcoded values
to_email = 'nnilayy.work@gmail.com'  # Replace with the recipient's email address
subject = 'Test Email with Attachments and Tracking Pixel'
body = '''Hello,

This is a test email sent from a Jupyter notebook. Please find the attachments and note that this email contains a tracking pixel.

Best regards,
Your Name'''


# If you have URLs to resume and cover letter PDFs, provide them here
# Replace with your actual URLs or leave as None if you don't want to include attachments
resume_url = 'https://drive.google.com/file/d/1mggvGbZYUdiZGnU3JUnIyRICjwElL0nB/view?pli=1'        # Replace with your resume URL
cover_letter_url = 'https://drive.google.com/file/d/1mggvGbZYUdiZGnU3JUnIyRICjwElL0nB/view?pli=1'  # Replace with your cover letter URL

# Call the function
send_email_with_attachments(to_email, subject, body, resume_url, cover_letter_url)


Email sent to nnilayy.work@gmail.com
